In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup
In this notebook, we will run SfM reconstruction from scratch on a set of images. We choose the [South-Building dataset](https://openaccess.thecvf.com/content_cvpr_2013/html/Hane_Joint_3D_Scene_2013_CVPR_paper.html) - we will download it later. First, we define some paths.

In [4]:
images = Path('/root/code/data/images/')

outputs = Path('outputs/sfm/')
sfm_pairs = outputs / 'pairs-netvlad.txt'
sfm_dir = outputs / 'sfm_superpoint+superglue'

retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

## Download the dataset
The dataset is simply a set of images. The intrinsic parameters will be extracted from the EXIF data and refined with SfM.

In [5]:
if not images.exists():
    !wget http://cvg.ethz.ch/research/local-feature-evaluation/South-Building.zip -P datasets/
    !unzip -q datasets/South-Building.zip -d datasets/

## Find image pairs via image retrieval
We extract global descriptors with NetVLAD and find for each image the most similar ones. For smaller dataset we can instead use exhaustive matching via `hloc/pairs_from_exhaustive.py`, which would find $\frac{n(n-1)}{2}$ images pairs.

In [6]:
retrieval_path = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=5)

[2023/11/20 13:01:11 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2023/11/20 13:01:11 hloc INFO] Found 710 images in root /root/code/data/images.
100%|████████████████| 529M/529M [01:28<00:00, 6.29MB/s]
100%|█████████████████| 710/710 [04:06<00:00,  2.88it/s]
[2023/11/20 13:06:56 hloc INFO] Finished exporting features.
[2023/11/20 13:06:56 hloc INFO] Extracting image pairs from a retrieval database.
[2023/11/20 13:07:05 hloc INFO] Found 3550 pairs.


## Extract and match local features

In [7]:
feature_path = extract_features.main(feature_conf, images, outputs)
match_path = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)

[2023/11/20 13:07:23 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2023/11/20 13:07:23 hloc INFO] Found 710 images in root /root/code/data/images.


Loaded SuperPoint model


100%|█████████████████| 710/710 [03:19<00:00,  3.56it/s]
[2023/11/20 13:10:43 hloc INFO] Finished exporting features.
[2023/11/20 13:10:43 hloc INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}


Loaded SuperGlue model ("outdoor" weights)


  0%|                          | 0/2110 [00:00<?, ?it/s]ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
  0%|                          | 0/2110 [00:00<?, ?it/s]


RuntimeError: DataLoader worker (pid(s) 189, 190, 192, 193) exited unexpectedly

## 3D reconstruction
Run COLMAP on the features and matches.

In [ ]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, feature_path, match_path)

## Visualization
We visualize some of the registered images, and color their keypoint by visibility, track length, or triangulated depth.

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='visibility', n=5)

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='track_length', n=5)

In [ ]:
visualization.visualize_sfm_2d(model, images, color_by='depth', n=5)